# SYNTHÈSE ANNUELLE

Suite à une réunion le 01/03/2024, la direction de la médiathèque souhaite qu'on lui communique une petite synthèse des chiffres-clés de l'année n-1, en se basant sur les principaux indicateurs. L'objectif final est de pouvoir fournir des chiffres aux élus qui brosse les grandes tendance de l'année passée par rapport à l'année d'avant (année n-2).
Pour ces raisons, l'idée est d'avoir pour chaque indicateur, une phrase contenant : 
* la valeur brut de l'année n-1
* la valeur brut de l'année n-2
* le pourcentage d'évolution de l'année n-1 par rapport à l'année n-2

**Quelles sont les informations demandés ?**

*ETABLISSEMENT*
* Nombre de jours ouverts

*ENTREES*
* Nombre d'entrées
* Nombre d'entrées le dimanche

*INSCRITS*
* Nombre d'inscrits
* Nombre de nouveaux inscrits
* Répartition inscrits roubaisien vs non-roubaisiens
* Répartition par type de carte
* Répartition par tranche d'âge

*PRÊTS*
* Nombre de prêts
* Nombre d'emprunteurs distincts
* Nombre de prêts par site
* Nombre de prêts par supports (livres,dvd,cd)
* Nombre de prêts par public (jeunesse ou adulte)

*CONNEXIONS ORDINATEURS*
* Nombre de connexions aux ordinateurs
* Nombre d'utilisateurs distincts des ordinateurs

*CONSULTATIONS DU SITE WEB*
* Nombre de visites du site web
* Nombre de visiteurs distincts du site web

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from kiblib.utils.db import DbConn
db_conn = DbConn().create_engine()

In [2]:
year_1 = datetime.today().isocalendar()[0] - 1
#year_2 = datetime.today().isocalendar()[0] - 2
year_2 = 2023
year_3 = 2018

In [3]:
# 2 fonctions essentielles

def PourcentageEvolution(va,vd):
    evolution = ((va-vd)/vd)*100
    evolution = round(evolution,0)
    evolution = int(evolution)
    return(f'{evolution}%')


def PourcentageRepartition(partie,ensemble):
    repartition = partie/ensemble*100
    repartition = round(repartition,0)
    repartition = int(repartition)
    return(f"{repartition}%")

# Entrées à la médiathèque

In [4]:
query = f"""SELECT datetime, entrees 
FROM statdb.stat_entrees
WHERE YEAR(`datetime`)={year_1}"""

entrees_year_1 = pd.read_sql(query,db_conn)

# Création d'une colonne date pour regrouper les jours uniques
entrees_year_1['date'] = pd.DatetimeIndex(entrees_year_1['datetime']).date

#Ajout d'une colonne day_name
entrees_year_1['nom_jour'] = pd.DatetimeIndex(entrees_year_1['datetime']).day_name(locale='fr_FR.utf8')

# Calcule nombre de jours ouverts
nb_jours_ouverts_year_1 = len(entrees_year_1.groupby('date'))

total_entrees_year_1 = entrees_year_1['entrees'].sum()

total_entrees_dimanche_year_1 = entrees_year_1[entrees_year_1['nom_jour']=='Dimanche']['entrees'].sum()

In [5]:
query = f"""SELECT datetime, entrees 
FROM statdb.stat_entrees
WHERE YEAR(`datetime`)={year_2}"""

entrees_year_2 = pd.read_sql(query,db_conn)

# Création d'une colonne date pour regrouper les jours uniques
entrees_year_2['date'] = pd.DatetimeIndex(entrees_year_2['datetime']).date

#Ajout d'une colonne day_name
entrees_year_2['nom_jour'] = pd.DatetimeIndex(entrees_year_2['datetime']).day_name(locale='fr_FR.utf8')

# Calcule nombre de jours ouverts
nb_jours_ouverts_year_2 = len(entrees_year_2.groupby('date'))

total_entrees_year_2 = entrees_year_2['entrees'].sum()

total_entrees_dimanche_year_2 = entrees_year_2[entrees_year_2['nom_jour']=='Dimanche']['entrees'].sum()

In [6]:
evolution_entrees = PourcentageEvolution(va=total_entrees_year_1,vd=total_entrees_year_2)

# Inscrits à La Grand Plage

In [7]:
query = f"""SELECT *
FROM statdb.stat_adherents sa 
WHERE YEAR(date_extraction) = '{year_1}'
AND date_extraction = (SELECT MAX(date_extraction) 
                       FROM statdb.stat_adherents sa2  
                       WHERE YEAR(date_extraction) = '{year_1}');"""

inscrits_year_1 = pd.read_sql(query,db_conn)

# Ajout d'une colonne tranche d'âge
inscrits_year_1.loc[inscrits_year_1['age']<=14,["tranche d'âge"]] = '0-14 ans'
inscrits_year_1.loc[inscrits_year_1['age']>=15,["tranche d'âge"]] = '15 ans et plus'

# Nombre d'inscrit par tranche d'âge
nb_inscrits_moins_de_14_ans_year_1 = len(inscrits_year_1[inscrits_year_1["tranche d'âge"]=='0-14 ans'])
nb_inscrits_15_ans_et_plus_year_1 = len(inscrits_year_1[inscrits_year_1["tranche d'âge"]=='15 ans et plus'])

# Total inscrits
total_inscrits_year_1 = len(inscrits_year_1)

# Pourcentage d'inscrits par tranche d'âge
pct_moins_de_14_ans_year_1 = PourcentageRepartition(partie=nb_inscrits_moins_de_14_ans_year_1,
                                                             ensemble=total_inscrits_year_1)

pct_15_ans_et_plus_year_1 = PourcentageRepartition(partie=nb_inscrits_15_ans_et_plus_year_1,
                                                            ensemble=total_inscrits_year_1)

# Total nouveaux inscrits
total_inscrits_nvx_year_1 = len(inscrits_year_1[inscrits_year_1['inscription_fidelite']==0])

# Pourcentage de nouveaux inscrits (moins d'1 an)
pct_inscrits_nvx_year_1 = PourcentageRepartition(partie=total_inscrits_nvx_year_1,
                                                   ensemble=total_inscrits_year_1)
pct_inscrits_nvx_year_1

# Création d'une colonne Roubaisien?
inscrits_year_1.loc[inscrits_year_1['geo_ville']=='ROUBAIX',['roubaisien?']] = 'OUI'

# Nombre d'inscrits roubaisiens
nb_inscrits_roubaisiens_year_1 = len(inscrits_year_1[inscrits_year_1['roubaisien?']=='OUI'])

# Liste des cartes médiathèques payantes
carte_mediatheque = ['MEDA','MEDB','MEDC','CSVT']

# Création d'une colonne type de carte
inscrits_year_1.loc[inscrits_year_1['inscription_code_carte'].isin(carte_mediatheque),['type_de_carte']] = 'Carte Médiathèque Plus'
inscrits_year_1.loc[inscrits_year_1['inscription_code_carte']=='BIBL',['type_de_carte']] = 'Carte Médiathèque (gratuite)'
inscrits_year_1.loc[inscrits_year_1['inscription_code_carte']=='CSLT',['type_de_carte']] = 'Carte de consultation sur place'

# Nombre d'inscrits par types de cartes
nb_inscrits_carte_med_year_1 = len(inscrits_year_1[inscrits_year_1['type_de_carte']=='Carte Médiathèque Plus'])
nb_inscrits_carte_bib_year_1 = len(inscrits_year_1[inscrits_year_1['type_de_carte']=='Carte Médiathèque (gratuite)'])
nb_inscrits_carte_consultation_year_1 = len(inscrits_year_1[inscrits_year_1['type_de_carte']=='Carte de consultation sur place'])

# Pourcentage d'inscrits par type de carte
pct_carte_med_year_1 = PourcentageRepartition(partie=nb_inscrits_carte_med_year_1,
                                                       ensemble=total_inscrits_year_1)

pct_carte_bib_year_1 = PourcentageRepartition(partie=nb_inscrits_carte_bib_year_1,
                                                       ensemble=total_inscrits_year_1)

pct_carte_consultation_year_1 = PourcentageRepartition(partie=nb_inscrits_carte_consultation_year_1,
                                                       ensemble=total_inscrits_year_1)

In [8]:
query = f"""SELECT *
FROM statdb.stat_adherents sa 
WHERE YEAR(date_extraction) = '{year_2}'
AND date_extraction = (SELECT MAX(date_extraction) 
                       FROM statdb.stat_adherents sa2  
                       WHERE YEAR(date_extraction) = '{year_2}');"""

inscrits_year_2 = pd.read_sql(query,db_conn)

# Ajout d'une colonne tranche d'âge
inscrits_year_2.loc[inscrits_year_2['age']<=14,["tranche d'âge"]] = '0-14 ans'
inscrits_year_2.loc[inscrits_year_2['age']>=15,["tranche d'âge"]] = '15 ans et plus'

# Nombre d'inscrit par tranche d'âge
nb_inscrits_moins_de_14_ans_year_2 = len(inscrits_year_2[inscrits_year_2["tranche d'âge"]=='0-14 ans'])
nb_inscrits_15_ans_et_plus_year_2 = len(inscrits_year_2[inscrits_year_2["tranche d'âge"]=='15 ans et plus'])

# Total inscrits
total_inscrits_year_2 = len(inscrits_year_2)

# Pourcentage d'inscrits par tranche d'âge
pct_moins_de_14_ans_year_2 = PourcentageRepartition(partie=nb_inscrits_moins_de_14_ans_year_2,
                                                             ensemble=total_inscrits_year_2)

pct_15_ans_et_plus_year_2 = PourcentageRepartition(partie=nb_inscrits_15_ans_et_plus_year_2,
                                                            ensemble=total_inscrits_year_2)

# Total nouveaux inscrits
total_inscrits_nvx_year_2 = len(inscrits_year_2[inscrits_year_2['inscription_fidelite']==0])

# Pourcentage de nouveaux inscrits (moins d'1 an)
pct_inscrits_nvx_year_2 = PourcentageRepartition(partie=total_inscrits_nvx_year_2,
                                                   ensemble=total_inscrits_year_2)
pct_inscrits_nvx_year_2

# Création d'une colonne Roubaisien?
inscrits_year_2.loc[inscrits_year_2['geo_ville']=='ROUBAIX',['roubaisien?']] = 'OUI'

# Nombre d'inscrits roubaisiens
nb_inscrits_roubaisiens_year_2 = len(inscrits_year_2[inscrits_year_2['roubaisien?']=='OUI'])

# Liste des cartes médiathèques payantes
carte_mediatheque = ['MEDA','MEDB','MEDC','CSVT']

# Création d'une colonne type de carte
inscrits_year_2.loc[inscrits_year_2['inscription_code_carte'].isin(carte_mediatheque),['type_de_carte']] = 'Carte Médiathèque Plus'
inscrits_year_2.loc[inscrits_year_2['inscription_code_carte']=='BIBL',['type_de_carte']] = 'Carte Médiathèque (gratuite)'
inscrits_year_2.loc[inscrits_year_2['inscription_code_carte']=='CSLT',['type_de_carte']] = 'Carte de consultation sur place'

# Nombre d'inscrits par types de cartes
nb_inscrits_carte_med_year_2 = len(inscrits_year_2[inscrits_year_2['type_de_carte']=='Carte Médiathèque Plus'])
nb_inscrits_carte_bib_year_2 = len(inscrits_year_2[inscrits_year_2['type_de_carte']=='Carte Médiathèque (gratuite)'])
nb_inscrits_carte_consultation_year_2 = len(inscrits_year_2[inscrits_year_2['type_de_carte']=='Carte de consultation sur place'])

# Pourcentage d'inscrits par type de carte
pct_carte_med_year_2 = PourcentageRepartition(partie=nb_inscrits_carte_med_year_2,
                                                       ensemble=total_inscrits_year_2)

pct_carte_bib_year_2 = PourcentageRepartition(partie=nb_inscrits_carte_bib_year_2,
                                                       ensemble=total_inscrits_year_2)

pct_carte_consultation_year_2 = PourcentageRepartition(partie=nb_inscrits_carte_consultation_year_2,
                                                       ensemble=total_inscrits_year_2)

In [9]:
# Evolution des inscrits
evolution_inscrits = PourcentageEvolution(va=total_inscrits_year_1,vd=total_inscrits_year_2)

# Les prêts

In [10]:
# On regroupe les différentes localisations de documents par espaces, dans une liste
espace_jeunesse_med = ('MED2A','MED2C')
autres_espaces_med = ('MED0B','MED0C','MED1A','MED2B','MED3A','MED3C','MED3E','MED3F','MED3H','MED3I','MED3K','MED3M','MED3O','MED3T')
tous_espaces_med = ('MED2A','MED2C''MED0B','MED0C','MED1A','MED2B','MED3A','MED3C','MED3E','MED3F','MED3H','MED3I','MED3K','MED3M','MED3O','MED3T')

In [11]:
query = f"""SELECT * 
FROM statdb.stat_issues si
WHERE YEAR(issuedate) = {year_1}"""

In [12]:
prets_year_1 = pd.read_sql(query,db_conn)

In [13]:
# Dataframe avec les prêts localisés en médiathèque
prets_med_year_1 = prets_year_1[prets_year_1['location'].isin(tous_espaces_med)]

In [14]:
# Nombre de prêts en espace JEUNESSE et dans les AUTRES ESPACES
nb_prets_jeunesse_med_year_1 = len(prets_med_year_1[prets_year_1['location'].isin(espace_jeunesse_med)])
nb_prets_pas_jeunesse_med_year_1 = len(prets_med_year_1[prets_year_1['location'].isin(autres_espaces_med)])

/tmp/ipykernel_21300/1183353713.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nb_prets_jeunesse_med_year_1 = len(prets_med_year_1[prets_year_1['location'].isin(espace_jeunesse_med)])
/tmp/ipykernel_21300/1183353713.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nb_prets_pas_jeunesse_med_year_1 = len(prets_med_year_1[prets_year_1['location'].isin(autres_espaces_med)])


In [15]:
# Nombre de prêts tous services confondus ET Nombre de prêts localisés à la médiathèque
total_prets_med_year_1 = len(prets_med_year_1)

In [16]:
repartitition_prets_jeunesse_med_year_1 = PourcentageRepartition(partie=nb_prets_jeunesse_med_year_1,
                                                              ensemble=total_prets_med_year_1)

repartitition_prets_pas_jeunesse_med_year_1 = PourcentageRepartition(partie=nb_prets_pas_jeunesse_med_year_1,
                                                              ensemble=total_prets_med_year_1)

In [17]:
# Nombre de prêts par site 
nb_prets_med_year_1 = len(prets_year_1[prets_year_1['branch']=='MED'])
nb_prets_bus_year_1 = len(prets_year_1[prets_year_1['branch']=='BUS'])

In [18]:
# Nombre de prêts total
total_prets_year_1 = len(prets_year_1)

In [19]:
#pourcentage de prêts médiathèque
pct_prets_med_year_1 = PourcentageRepartition(partie=nb_prets_med_year_1,ensemble=total_prets_year_1)

#pourcentage de prêts bus
pct_prets_bus_year_1 = PourcentageRepartition(partie=nb_prets_bus_year_1,ensemble=total_prets_year_1)

In [20]:
# Nombre de prêts par type de support
nb_prets_livres_year_1 = len(prets_year_1[prets_year_1['itemtype']=='LI'])
nb_prets_dvd_year_1 = len(prets_year_1[prets_year_1['itemtype']=='VD'])

#pourcentage de prêts livres
pct_prets_livres_year_1 = PourcentageRepartition(partie=nb_prets_livres_year_1,ensemble=total_prets_year_1)

#pourcentage de prêts dvd
pct_prets_dvd_year_1 = PourcentageRepartition(partie=nb_prets_dvd_year_1,ensemble=total_prets_year_1)

In [21]:
# Conversion de la colonne AGE en INT
prets_med_year_1['age'] = pd.to_numeric(prets_med_year_1['age'],
                                        downcast="integer",
                                        errors="coerce")


/tmp/ipykernel_21300/1042478012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prets_med_year_1['age'] = pd.to_numeric(prets_med_year_1['age'],


In [22]:
# Nombre de prêt par tranche d'âge des emprunteurs
prets_med_year_1.loc[prets_med_year_1["age"]<=14,["Tranche d'âge"]] = "0-14 ans"
prets_med_year_1.loc[prets_med_year_1["age"]>14,["Tranche d'âge"]] = "15 ans et plus"

nb_prets_med_zero_quatorze_ans = len(prets_med_year_1[prets_med_year_1["Tranche d'âge"]=="0-14 ans"])
nb_prets_med_quinze_ans_et_plus = len(prets_med_year_1[prets_med_year_1["Tranche d'âge"]=="15 ans et plus"])

# Pourcentage de prêts par tranche d'âge des emprunteurs
pct_jeunes_prets_med_year_1 = PourcentageRepartition(partie=nb_prets_med_zero_quatorze_ans,
                                                       ensemble=total_prets_med_year_1)

pct_adultes_prets_med_year_1 = PourcentageRepartition(partie=nb_prets_med_quinze_ans_et_plus,
                                                       ensemble=total_prets_med_year_1)

/tmp/ipykernel_21300/697466818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prets_med_year_1.loc[prets_med_year_1["age"]<=14,["Tranche d'âge"]] = "0-14 ans"


In [23]:
query = f"""SELECT * 
FROM statdb.stat_issues si
WHERE YEAR(issuedate) = {year_2}"""

prets_year_2 = pd.read_sql(query,db_conn)

# Dataframe avec les prêts localisés en médiathèque
prets_med_year_2 = prets_year_2[prets_year_2['location'].isin(tous_espaces_med)]

# Nombre de prêts en espace JEUNESSE et dans les AUTRES ESPACES
nb_prets_jeunesse_med_year_2 = len(prets_med_year_2[prets_year_2['location'].isin(espace_jeunesse_med)])
nb_prets_pas_jeunesse_med_year_2 = len(prets_med_year_2[prets_year_2['location'].isin(autres_espaces_med)])

# Nombre de prêts tous services confondus ET Nombre de prêts localisés à la médiathèque
total_prets_med_year_2 = len(prets_med_year_2)

repartitition_prets_jeunesse_med_year_2 = PourcentageRepartition(partie=nb_prets_jeunesse_med_year_2,
                                                              ensemble=total_prets_med_year_2)

repartitition_prets_pas_jeunesse_med_year_2 = PourcentageRepartition(partie=nb_prets_pas_jeunesse_med_year_2,
                                                              ensemble=total_prets_med_year_2)

# Nombre de prêts par site 
nb_prets_med_year_2 = len(prets_year_2[prets_year_2['branch']=='MED'])
nb_prets_bus_year_2 = len(prets_year_2[prets_year_2['branch']=='BUS'])

# Nombre de prêts total
total_prets_year_2 = len(prets_year_2)

#pourcentage de prêts médiathèque
pct_prets_med_year_2 = PourcentageRepartition(partie=nb_prets_med_year_2,ensemble=total_prets_year_2)

#pourcentage de prêts bus
pct_prets_bus_year_2 = PourcentageRepartition(partie=nb_prets_bus_year_2,ensemble=total_prets_year_2)

# Nombre de prêts par type de support
nb_prets_livres_year_2 = len(prets_year_2[prets_year_2['itemtype']=='LI'])
nb_prets_dvd_year_2 = len(prets_year_2[prets_year_2['itemtype']=='VD'])

#pourcentage de prêts livres
pct_prets_livres_year_2 = PourcentageRepartition(partie=nb_prets_livres_year_2,ensemble=total_prets_year_2)

#pourcentage de prêts dvd
pct_prets_dvd_year_2 = PourcentageRepartition(partie=nb_prets_dvd_year_2,ensemble=total_prets_year_2)

# Conversion de la colonne AGE en INT
prets_med_year_2['age'] = pd.to_numeric(prets_med_year_2['age'],
                                        downcast="integer",
                                        errors="coerce")


# Nombre de prêt par tranche d'âge des emprunteurs
prets_med_year_2.loc[prets_med_year_2["age"]<=14,["Tranche d'âge"]] = "0-14 ans"
prets_med_year_2.loc[prets_med_year_2["age"]>14,["Tranche d'âge"]] = "15 ans et plus"

nb_prets_med_zero_quatorze_ans = len(prets_med_year_2[prets_med_year_2["Tranche d'âge"]=="0-14 ans"])
nb_prets_med_quinze_ans_et_plus = len(prets_med_year_2[prets_med_year_2["Tranche d'âge"]=="15 ans et plus"])

# Pourcentage de prêts par tranche d'âge des emprunteurs
pct_jeunes_prets_med_year_2 = PourcentageRepartition(partie=nb_prets_med_zero_quatorze_ans,
                                                       ensemble=total_prets_med_year_2)

pct_adultes_prets_med_year_2 = PourcentageRepartition(partie=nb_prets_med_quinze_ans_et_plus,
                                                       ensemble=total_prets_med_year_2)

# Evolution des prêts
evolultion_prets = PourcentageEvolution(va=total_prets_year_2,vd=total_prets_year_2)

/tmp/ipykernel_21300/1877173782.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nb_prets_jeunesse_med_year_2 = len(prets_med_year_2[prets_year_2['location'].isin(espace_jeunesse_med)])
/tmp/ipykernel_21300/1877173782.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nb_prets_pas_jeunesse_med_year_2 = len(prets_med_year_2[prets_year_2['location'].isin(autres_espaces_med)])
/tmp/ipykernel_21300/1877173782.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prets_med_year_2['age'] = pd.to_numeric(prets_med_year_2['age'],
/tmp/ipykernel_21300/1877173782.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
# Evolution des prêts
evolultion_prets = PourcentageEvolution(va=total_prets_year_1,vd=total_prets_year_2)

# Réservations

In [25]:
query = f"""SELECT * 
FROM statdb.stat_reserves sr
WHERE YEAR(reservedate) = 
{year_1}"""

resas_year_1 = pd.read_sql(query,db_conn)

nb_resas_distincts_year_1 = resas_year_1['borrowernumber'].nunique()

total_resas_year_1 = len(resas_year_1)

In [26]:
query = f"""SELECT * 
FROM statdb.stat_reserves sr
WHERE YEAR(reservedate) = {year_2}"""

resas_year_2 = pd.read_sql(query,db_conn)
total_resas_year_2 = len(resas_year_2)

total_resas_year_2 = len(resas_year_2)

nb_resas_distincts_year_2 = resas_year_2['borrowernumber'].nunique()

In [27]:
evolution_resas = PourcentageEvolution(va=total_resas_year_1,vd=total_resas_year_2)
evolution_resas_distincts = PourcentageEvolution(va=nb_resas_distincts_year_1,vd=nb_resas_distincts_year_2)

# Les connexions aux ordis

In [28]:
query = f"""SELECT *
FROM statdb.stat_webkiosk sw 
WHERE YEAR(heure_deb) = '{year_1}'"""

ordis_year_1 = pd.read_sql(query,db_conn)

# Nombre total de connexions aux ordinateurs
total_ordis_year_1 = len(ordis_year_1)

# Nombre de connexions ordis par des utilisateurs provenant de roubaix
total_ordis_rbx_year_1 = len(ordis_year_1[ordis_year_1['ville']=='ROUBAIX'])

# Pourcentage de connexions ordis par des utilisateurs provenant de Roubaix
pct_ordis_year_1 = PourcentageRepartition(partie=total_ordis_rbx_year_1,ensemble=total_ordis_year_1)

In [29]:
query = f"""SELECT *
FROM statdb.stat_webkiosk sw 
WHERE YEAR(heure_deb) = '{year_2}'"""

ordis_year_2 = pd.read_sql(query,db_conn)

# Nombre total de connexions aux ordinateurs
total_ordis_year_2 = len(ordis_year_2)

# Nombre de connexions ordis par des utilisateurs provenant de roubaix
total_ordis_rbx_year_2 = len(ordis_year_2[ordis_year_2['ville']=='ROUBAIX'])

# Pourcentage de connexions ordis par des utilisateurs provenant de Roubaix
pct_ordis_year_2 = PourcentageRepartition(partie=total_ordis_rbx_year_2,ensemble=total_ordis_year_2)

In [30]:
evolution_ordis = PourcentageEvolution(va=total_ordis_year_1,vd=total_ordis_year_2)

# Connexions wifi

In [31]:
query = f"""SELECT *
FROM statdb.stat_wifi si 
WHERE YEAR(start_wifi) = '{year_1}'"""

wifi_year_1 = pd.read_sql(query,db_conn)

# Nombre de connexions wifi
total_wifi_year_1 = len(wifi_year_1)

In [32]:
query = f"""SELECT *
FROM statdb.stat_wifi si 
WHERE YEAR(start_wifi) = '{year_2}'"""

wifi_year_2 = pd.read_sql(query,db_conn)

# Nombre de connexions wifi
total_wifi_year_2 = len(wifi_year_2)

In [33]:
evolution_wifi = PourcentageEvolution(va=total_wifi_year_1,vd=total_wifi_year_2)

# Sites web

## Portail

In [34]:
query = f"""SELECT * 
FROM statdb.stat_web2 sw
WHERE YEAR(`date`) = {year_1}"""

web_year_1 = pd.read_sql(query,db_conn)

nb_visites_web_year_1 = web_year_1['visites'].sum()

In [35]:
query = f"""SELECT * 
FROM statdb.stat_web2 sw
WHERE YEAR(`date`) = {year_2}"""

web_year_2 = pd.read_sql(query,db_conn)

nb_visites_web_year_2 = web_year_2['visites'].sum()
nb_visites_web_year_2

182074

In [36]:
evolution_web = PourcentageEvolution(va=nb_visites_web_year_1,vd=nb_visites_web_year_2)

# Bn-r

In [37]:
bnr = pd.read_excel('../stats_bnr_synthese.xlsx')

In [38]:
#Nombre de visites par an
nb_visites_bnr_year_1 = bnr[bnr['annee']==year_1]['nb_visites'].sum()
nb_visites_bnr_year_2 = bnr[bnr['annee']==year_2]['nb_visites'].sum()


#evolution
evolution_bnr = PourcentageEvolution(va=nb_visites_bnr_year_1,vd=nb_visites_bnr_year_2)

In [39]:
def MakeSpace(x):
    x = format(x,',').replace(',',' ')
    return(x)

In [40]:
nums = [nb_inscrits_15_ans_et_plus_year_1,
        nb_inscrits_15_ans_et_plus_year_2,
        nb_inscrits_carte_bib_year_1,
        nb_inscrits_carte_bib_year_2,
        nb_inscrits_carte_consultation_year_1,
        nb_inscrits_carte_consultation_year_2,
        nb_inscrits_carte_med_year_1,
        nb_inscrits_carte_med_year_2,
        nb_inscrits_moins_de_14_ans_year_1,
        nb_inscrits_moins_de_14_ans_year_2,
        nb_inscrits_roubaisiens_year_1,
        nb_inscrits_roubaisiens_year_2,
        nb_prets_bus_year_1,
        nb_prets_bus_year_2,
        nb_prets_dvd_year_1,
        nb_prets_dvd_year_2,
        nb_prets_jeunesse_med_year_1,
        nb_prets_jeunesse_med_year_2,
        nb_prets_livres_year_1,
        nb_prets_livres_year_2,
        nb_prets_med_quinze_ans_et_plus,
        nb_prets_med_year_1,
        nb_prets_med_year_2,
        nb_prets_pas_jeunesse_med_year_1,
        nb_prets_pas_jeunesse_med_year_2
       ]

# Résultats

In [42]:
print(f"SYNTHÈSE ANNUELLE POUR L'ANNÉE {year_1}")
print(100*'-')
print("\n")

print("LES INSCRITS\n")
print(f"En {year_1}, on compte {total_inscrits_year_1} personnes inscrites (dont {pct_inscrits_nvx_year_1} de nouveaux inscrits).\nLe nombre d'inscrits était de {total_inscrits_year_2} en {year_2}, soit une évolution de {evolution_inscrits} du nombre d'inscrits en {year_1}")
print(f"Parmi les adhérents, on compte en {year_1} {nb_inscrits_roubaisiens_year_1} roubaisiens, soit {PourcentageRepartition(partie=nb_inscrits_roubaisiens_year_1,ensemble=total_inscrits_year_1)} du total d'inscrits. Contre {PourcentageRepartition(partie=nb_inscrits_roubaisiens_year_2,ensemble=total_inscrits_year_2)} en {year_2}")
print(f"En {year_1} toujours: \n {nb_inscrits_carte_med_year_1} adhérents ont l'abonnement Médiathèque + (soit {pct_carte_med_year_1} du nombre total d'adhérents)\n {nb_inscrits_carte_bib_year_1} ont l'abonnement médiathèque simple (soit {pct_carte_bib_year_1}) et\n {nb_inscrits_carte_consultation_year_1} l'abonnement de consultation sur place (soit {pct_carte_consultation_year_1})")
print(f" - Les 0-14 ans : {nb_inscrits_moins_de_14_ans_year_1} inscrits (soit {pct_moins_de_14_ans_year_1} du total inscrits)")
print(f" - Les 15 ans et plus : {nb_inscrits_15_ans_et_plus_year_1} inscrits (soit {pct_15_ans_et_plus_year_1} du total inscrit)")
print("\n")

print("LES ENTRÉES\n")
print(f"En {year_1}, la médiathèque a été ouverte {nb_jours_ouverts_year_1} jours")
print(f"On compte {total_entrees_year_1} entrées ,contre {total_entrees_year_2} entrées en {year_2},soit une évolution de {evolution_entrees}")
print(f"Par ailleurs, la médiathèque comptabilise {total_entrees_dimanche_year_1} entrées en {year_1} les dimanches, ce qui représente {np.round(total_entrees_dimanche_year_1/total_entrees_year_1*100,0).astype(int)}% des entrées totales en {year_1}")
print("\n")

print("LES PRÊTS\n")
print(f"En {year_1}, on compte {total_prets_year_1} prêts à La Grand-Plage contre {total_prets_year_2} prêts en {year_2} soit une évolution de {evolultion_prets}")
print(f"Parmi ces {total_prets_year_1}, on compte :\n{nb_prets_med_year_1} prêts médiathèque (soit {pct_prets_med_year_1} des prêts totaux) et\n{nb_prets_bus_year_1} de prêts au Zèbre (soit {pct_prets_bus_year_1} des prêts totaux)")
print(f"Les prêts de documents localisé à l'espace jeunesse représentent {repartitition_prets_jeunesse_med_year_1} du total de prêts à la médiathèque en {year_1}")
print(f"Par ailleurs les livres représentent {pct_prets_livres_year_1} des prêts totaux (soit {nb_prets_livres_year_1}) tandis que les dvd représentent {pct_prets_dvd_year_1}")
print(f"{nb_prets_jeunesse_med_year_1} prêts jeunesse soit {repartitition_prets_jeunesse_med_year_1} du total de prêt de documents localisés à la médiathèque")
print(f"{nb_prets_pas_jeunesse_med_year_1} prêts adultes (hors collectivités) soit {repartitition_prets_pas_jeunesse_med_year_1} du total de prêt de documents localisés à la médiathèque")
print("\n")

print("LES RÉSAS\n")
print(f"En {year_1}, on compte {total_resas_year_1} réservations contre {total_resas_year_2}, soit une évolution de {evolution_resas} en {year_2})")
print(f"En {year_1}, on compte {nb_resas_distincts_year_1} réserveurs distincts, contre {nb_resas_distincts_year_2} en {year_2} soit une évolution de {evolution_resas_distincts}")
print("\n")

print("LES CONNEXIONS AUX ORDINATEURS\n")
print(f"En {year_1}, on compte {total_ordis_year_1} connexions aux postes informatiques, contre {total_ordis_year_2} connexions en  {year_2}, soit une évolution de {evolution_ordis}")
print(f"En 2023 toujours, {pct_ordis_year_1} des connexions provenait d'usagers habitant à Roubaix, contre {pct_ordis_year_2} en {year_2}")
print("\n")

print(f"LE NOMBRE DE VISITES DES SITES WEBS\n")
print(f"Nombre de visites du portail médiathèque en {year_1} : {nb_visites_web_year_1} contre {nb_visites_web_year_2} visites en {year_2} soit une évolution de {evolution_web}")
print(f"Nombre de visites de la bn-r en {year_1}: {nb_visites_bnr_year_1} contre {nb_visites_bnr_year_2} visites en {year_2} soit une évolution de {evolution_bnr}")

SYNTHÈSE ANNUELLE POUR L'ANNÉE 2024
----------------------------------------------------------------------------------------------------


LES INSCRITS

En 2024, on compte 15477 personnes inscrites (dont 42% de nouveaux inscrits).
Le nombre d'inscrits était de 15033 en 2023, soit une évolution de 3% du nombre d'inscrits en 2024
Parmi les adhérents, on compte en 2024 11005 roubaisiens, soit 71% du total d'inscrits. Contre 72% en 2023
En 2024 toujours: 
 781 adhérents ont l'abonnement Médiathèque + (soit 5% du nombre total d'adhérents)
 12806 ont l'abonnement médiathèque simple (soit 83%) et
 1436 l'abonnement de consultation sur place (soit 9%)
 - Les 0-14 ans : 5343 inscrits (soit 35% du total inscrits)
 - Les 15 ans et plus : 9843 inscrits (soit 64% du total inscrit)


LES ENTRÉES

En 2024, la médiathèque a été ouverte 283 jours
On compte 258427 entrées ,contre 232319 entrées en 2023,soit une évolution de 11%
Par ailleurs, la médiathèque comptabilise 15580 entrées en 2024 les dimanche